# Text Generation

Exploring raw text generation using llama-jax with and without KV caching.

# Setup

In [1]:
from contextlib import contextmanager
from sys import stdout
from time import time_ns, perf_counter_ns as timer

import jax
from jax import numpy as jnp
from jax import random

import llama_jax as ll

In [2]:
print(f"Available devices: {jax.devices()}")

Metal device set to: Apple M1 Max

systemMemory: 64.00 GB
maxCacheSize: 24.00 GB

Available devices: [METAL(id=0)]


W0000 00:00:1740264609.434287 75820107 mps_client.cc:510] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!
I0000 00:00:1740264609.444675 75820107 service.cc:145] XLA service 0x1295338e0 initialized for platform METAL (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1740264609.444684 75820107 service.cc:153]   StreamExecutor device (0): Metal, <undefined>
I0000 00:00:1740264609.445960 75820107 mps_client.cc:406] Using Simple allocator.
I0000 00:00:1740264609.445972 75820107 mps_client.cc:384] XLA backend will use up to 51539214336 bytes on device 0 for SimpleAllocator.


In [3]:
@contextmanager
def tps_report(n_tokens: int):
    start_time = timer()

    yield
    
    stdout.write("\n\n")
    
    duration = (timer() - start_time) / 1000000000
    tps = n_tokens / duration
    
    print(f"Generated {n_tokens} tokens in {duration:0.1f} s ({tps:0.1f} tps)")

# Configure

In [4]:
# Parameters
max_tokens = 20
seed = time_ns()

In [5]:
# Configure model
key = random.key(seed)
config = ll.checkpoint.load_config("Llama3.2-3B")
params = ll.checkpoint.load_parameters(config)
tokenizer = ll.checkpoint.load_tokenizer(config)
model = ll.model.create(config, params)

# Prompt

In [6]:
prompt = "I like traveling by train because"

# Tokenize

In [7]:
token_ids = tokenizer.encode(prompt)
token_ids

Array([[128000,     40,   1093,  21646,    555,   5542,   1606]], dtype=int32)

# Without KV Cache

In [8]:
with tps_report(max_tokens):    
    
    # Print input tokens
    stdout.write(prompt)
    
    # Process all tokens on first pass
    x = token_ids
    
    for _ in range(max_tokens):
        # Transform tokens into logits
        logits = ll.model.forward(config, model, x)
        
        # Sample next token
        next_token_id, key = ll.model.next_token(logits, key)
        
        # Print next token
        stdout.write(tokenizer.decode(next_token_id)[0])
    
        # Process all tokens on next pass
        x = jnp.concat([x, next_token_id], axis=-1)

I like traveling by train because I can enjoy the scenery. I like to travel by train because I can take a lot of photos

Generated 20 tokens in 22.1 s (0.9 tps)


# With KV Cache

In [9]:
with tps_report(max_tokens):    
    
    # Print input tokens
    stdout.write(prompt)
    
    # Initialize cache
    kv_cache = ll.kv_cache.create(config)
    
    # Process all tokens on first pass
    x = token_ids
    
    for _ in range(max_tokens):
        # Transform tokens into logits
        logits, kv_cache = ll.model.forward(config, model, x, kv_cache=kv_cache)
        
        # Sample next token
        next_token_id, key = ll.model.next_token(logits, key)
        
        # Print next token
        stdout.write(tokenizer.decode(next_token_id)[0])
    
        # Process generated token on next pass
        x = next_token_id

I like traveling by train because I can see the countryside and have a window seat. I don’t like the noise of the train

Generated 20 tokens in 16.5 s (1.2 tps)
